In [14]:
# Core data and analysis tools
import pandas as pd
import numpy as np
from scipy.stats import pearsonr, chi2_contingency 

# Visualization tools
import matplotlib.pyplot as plt
import seaborn as sns

In [15]:
df = pd.read_csv("sleep_health.csv")
df.head()

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,NaN
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea


In [16]:
contingency_table = pd.crosstab(df['Gender'], df['Sleep Disorder'])
chi2, p, dof, expected = chi2_contingency(contingency_table)
print(f"Chi-square statistic: {chi2:.2f}, p-value:{p:.4f}")

Chi-square statistic: 24.91, p-value:0.0000


In [17]:
pd.crosstab(df['Gender'], df['Sleep DIsorder'])

KeyError: 'Sleep DIsorder'